In [7]:
import pandas as pd
import pickle 
import sys
sys.path.append("/Ray_finrl/findrl_ray/finenv")
sys.path.append("./FinRL")
import finrl
from finenv.env_stocktrading import StockTradingEnv
from finenv.preprocessors import data_split
from finenv.save_model import upload_files
import psutil
import ray
from datetime import datetime
ray._private.utils.get_system_memory = lambda: psutil.virtual_memory().total
from ray.tune.registry import register_env
from gymnasium.wrappers import EnvCompatibility

# load the DataFrame from a pickle file
#df = pd.read_pickle('dataset/nasdaq74_proc_test.pkl')
train = pd.read_csv('dataset/train_data.csv')
train = train.set_index(train.columns[0])
train.index.names = ['']

In [8]:
INDICATORS = ['macd','boll_ub','boll_lb','rsi_30','cci_30','dx_30','close_30_sma','close_60_sma']

In [9]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 82, State Space: 821


In [10]:
train.head()
train.dtypes

date             object
tic              object
open            float64
high            float64
low             float64
close           float64
volume          float64
day             float64
macd            float64
boll_ub         float64
boll_lb         float64
rsi_30          float64
cci_30          float64
dx_30           float64
close_30_sma    float64
close_60_sma    float64
vix             float64
turbulence      float64
dtype: object

In [11]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

In [12]:
def env_creator(env_config):
    # env_config is passed as {} and defaults are set here
    df = env_config.get('df', train)
    hmax = env_config.get('hmax', 200)
    initial_amount = env_config.get('initial_amount', 1000000)
    num_stock_shares = env_config.get('num_stock_shares', [0] * stock_dimension)
    buy_cost_pct = env_config.get('buy_cost_pct', buy_cost_list)
    sell_cost_pct = env_config.get('sell_cost_pct', sell_cost_list)
    state_space = env_config.get('state_space', 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension)
    stock_dim = env_config.get('stock_dim', stock_dimension)
    tech_indicator_list = env_config.get('tech_indicator_list', INDICATORS)
    action_space = env_config.get('action_space', stock_dimension)
    reward_scaling = env_config.get('reward_scaling', 1e-2)

    return EnvCompatibility(StockTradingEnv(
        df=df,
        hmax=hmax,
        initial_amount=initial_amount,
        num_stock_shares=num_stock_shares,
        buy_cost_pct=buy_cost_pct,
        sell_cost_pct=sell_cost_pct,
        state_space=state_space,
        stock_dim=stock_dim,
        tech_indicator_list=tech_indicator_list,
        action_space=action_space,
        reward_scaling=reward_scaling
    ))


register_env("finrl", env_creator)

AttributeError: 'str' object has no attribute 'get'

In [13]:
from ray.rllib.agents import ppo
ray.shutdown()
#ray.init(num_cpus=45,num_gpus=4,dashboard_port=8300)
print(f"ray is being initialized")

ray is being initialized


In [14]:
from ray import tune
config={
        "env": "finrl",
            "kl_coeff": 1.0,
            "num_workers": 4,
            "num_cpus": 1,  # number of CPUs to use per trial
            "num_gpus": 0,  # number of GPUs to use per trial
            # These params are tuned from a fixed starting value.
            "lambda": 0.95,
            "clip_param": 0.2,
            "lr": 1e-4,
            # These params start off randomly drawn from a set.
            "num_sgd_iter": tune.choice([10, 20, 30]),
            "sgd_minibatch_size": tune.choice([128, 512, 2048]),
            "train_batch_size": tune.choice([10000, 20000, 40000]),
        }


In [15]:
tune.run("PPO", 
    config=config, 
    checkpoint_freq=1,
    keep_checkpoints_num=None,
    checkpoint_at_end=True,
    stop = { #"timesteps_total": NUMBER_OF_TIMESTEPS_TO_SAMPLE,
             "training_iteration": 5
           },
    local_dir='model/'
)

2023-04-03 23:08:18,194	INFO tune.py:922 -- Initializing Ray automatically.For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run`.
2023-04-03 23:08:24,373	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8266 


2023-04-03 23:08:26,285	INFO algorithm_config.py:2899 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2023-04-03 23:08:26,286	INFO algorithm_config.py:2899 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
(PPO pid=2157) 2023-04-03 23:08:34,358	WARNING algorithm_config.py:596 -- Cannot create PPOConfig from given `config_dict`! Property num_cpus not supported.
(PPO pid=2157) 2023-04-03 23:08:34,358	INFO algorithm_config.py:2899 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to se

Trial name,trial_id
PPO_finrl_624c1_00000,624c1_00000


(PPO pid=2157) 2023-04-03 23:08:42,539	ERROR actor_manager.py:496 -- Ray error, taking actor 1 out of service. The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=2289, ip=172.28.231.77, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7fef367d6d60>)
(PPO pid=2157)   File "/home/ga_aiot/anaconda3/envs/finrl/lib/python3.8/site-packages/gymnasium/envs/registration.py", line 569, in make
(PPO pid=2157)     _check_version_exists(ns, name, version)
(PPO pid=2157)   File "/home/ga_aiot/anaconda3/envs/finrl/lib/python3.8/site-packages/gymnasium/envs/registration.py", line 219, in _check_version_exists
(PPO pid=2157)     _check_name_exists(ns, name)
(PPO pid=2157)   File "/home/ga_aiot/anaconda3/envs/finrl/lib/python3.8/site-packages/gymnasium/envs/registration.py", line 197, in _check_name_exists
(PPO pid=2157)     raise error.NameNotFound(
(PPO pid=2157) gymnasium.error.NameNotFound: Environment finrl doesn't exist. 
(PPO pid=

TuneError: ('Trials did not complete', [PPO_finrl_624c1_00000])